# 📈 AV Simulation Visualization Examples

Create interactive visualizations of simulation data.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aanshshah/av-simulation/blob/main/examples/notebooks/04_visualization_examples.ipynb)

## Setup

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
import os
import sys

base_path = '/content' if ('/content' in os.getcwd() or 'COLAB_GPU' in os.environ) else os.getcwd()
repo_path = os.path.join(base_path, 'av-simulation')
sim_path = os.path.join(repo_path, 'src')

for path_entry in (repo_path, sim_path):
    if os.path.exists(path_entry) and path_entry not in sys.path:
        sys.path.insert(0, path_entry)

try:
    plt.style.use('seaborn-v0_8')
except Exception:
    try:
        plt.style.use('seaborn')
    except Exception:
        print('⚠️ Seaborn style not available, using default')

colors = {
    'ego': '#2E8B57',
    'other': '#FF6347',
    'road': '#808080'
}

sample_candidates = [
    os.path.join(repo_path, 'examples', 'notebooks', 'data', 'sample_ego_run.csv'),
    os.path.join(base_path, 'examples', 'notebooks', 'data', 'sample_ego_run.csv'),
    'examples/notebooks/data/sample_ego_run.csv'
]
sample_path = next((p for p in sample_candidates if os.path.exists(p)), None)

if sample_path:
    viz_df = pd.read_csv(sample_path)
    print(f'✅ Loaded sample data from {sample_path}')
else:
    print('⚠️ Sample data not found; generating synthetic fallback data')
    t = np.linspace(0, 10, 100)
    viz_df = pd.DataFrame({
        'timestamp': t,
        'speed': 20 + 5 * np.sin(t) + np.random.normal(0, 1, len(t)),
        'acceleration': np.gradient(20 + 5 * np.sin(t), t),
        'pos_x': np.linspace(0, 100, len(t)),
        'pos_y': 4 * np.sin(t / 2)
    })

globals()['viz_df'] = viz_df
globals()['colors'] = colors


## Create Plots

In [ ]:
# Sample data visualization
df = globals().get('viz_df')
colors = globals().get('colors', {})

if df is None or df.empty:
    print('❌ No data loaded for visualisation')
else:
    fig = plt.figure(figsize=(14, 10))

    ax1 = fig.add_subplot(2, 2, 1)
    ax1.plot(df['timestamp'], df['speed'], color=colors.get('ego', '#2E8B57'), linewidth=2)
    ax1.set_title('Vehicle Speed Profile')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Speed (m/s)')

    ax2 = fig.add_subplot(2, 2, 2)
    ax2.scatter(df['pos_x'], df['pos_y'], c=df['speed'], cmap='viridis', s=30)
    ax2.set_title('Trajectory Colored by Speed')
    ax2.set_xlabel('X Position (m)')
    ax2.set_ylabel('Y Position (m)')
    ax2.axis('equal')

    ax3 = fig.add_subplot(2, 2, 3)
    ax3.plot(df['timestamp'], df['acceleration'], color='#FF8C00', linewidth=2)
    ax3.set_title('Acceleration Over Time')
    ax3.set_xlabel('Time (s)')
    ax3.set_ylabel('Acceleration (m/s²)')

    ax4 = fig.add_subplot(2, 2, 4)
    ax4.hist(df['speed'], bins=15, color='#6495ED', edgecolor='black')
    ax4.set_title('Speed Distribution')
    ax4.set_xlabel('Speed (m/s)')
    ax4.set_ylabel('Frequency')

    fig.tight_layout()
    plt.show()

    if df.shape[0] >= 5:
        sample_trace = go.Scatter(
            x=df['timestamp'], y=df['speed'],
            mode='lines+markers',
            name='Speed (m/s)',
            line=dict(color=colors.get('ego', '#2E8B57'), width=2)
        )
        accel_trace = go.Bar(
            x=df['timestamp'], y=df['acceleration'],
            name='Acceleration (m/s²)',
            marker=dict(color='#FFB347', opacity=0.6)
        )

        fig_plotly = go.Figure(data=[sample_trace, accel_trace])
        fig_plotly.update_layout(
            title='Interactive Speed & Acceleration Overview',
            xaxis_title='Time (s)',
            yaxis_title='Magnitude',
            template='plotly_white'
        )
        fig_plotly.show()

    if {'pos_x', 'pos_y'} <= set(df.columns):
        heat_df = df.copy()
        heat_df['density'] = np.linspace(0, 1, len(heat_df))
        fig_heat = px.scatter(
            heat_df,
            x='pos_x', y='pos_y',
            color='density',
            title='Trajectory Heatmap',
            color_continuous_scale='Turbo'
        )
        fig_heat.update_traces(marker=dict(size=6))
        fig_heat.update_layout(xaxis_title='X Position (m)', yaxis_title='Y Position (m)')
        fig_heat.show()
